# Training TopKSAE Model

This notebook demonstrates how to:
1. Load a language model and dataset
2. Save activations from a specific layer
3. Train a TopK Sparse Autoencoder (TopKSAE) on those activations using the new `SaeTrainer` composite class
4. Save the trained TopKSAE model

The training uses overcomplete's `train_sae` functions via the `SaeTrainer` composite class, which is automatically available on all SAE instances via `sae.trainer`.

All files (trained TopKSAE model, training metadata, activations) will be saved under `store/{model_id}/` for organized, model-specific storage.


In [22]:
# Setup and imports
%load_ext autoreload
%autoreload 2

import torch
import os
from pathlib import Path
from datetime import datetime

# Silence Hugging Face tokenizers fork/parallelism warnings
os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")

from amber.store.store import LocalStore
from amber.adapters import TextDataset
from amber.core.language_model import LanguageModel
from amber.mechanistic.sae.modules.topk_sae import TopKSae, TopKSAETrainingConfig

print("✅ Imports completed")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
✅ Imports completed


In [23]:
# Configuration
MODEL_ID = "sshleifer/tiny-gpt2"  # Small model for quick experimentation
HF_DATASET = "roneneldan/TinyStories"
DATA_SPLIT = "train"
TEXT_FIELD = "text"
DATA_LIMIT = 1000  # Number of text samples to use
MAX_LENGTH = 64  # Maximum sequence length
BATCH_SIZE_SAVE = 16  # Batch size for saving activations
BATCH_SIZE_TRAIN = 32  # Batch size for SAE training

# TopKSAE configuration
TOP_K = 8  # Number of top activations to keep (sparsity parameter)

# Choose which layer to hook - you can inspect available layers with model.layers.print_layer_names()
LAYER_SIGNATURE = 'gpt2lmheadmodel_transformer_h_0_attn_c_attn'  # Attention layer (better activations)

# Storage locations - will be updated after model loading to use model_id
STORE_DIR = Path("store")
CACHE_DIR = Path("store/cache")
RUN_ID = f"topk_sae_training_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
# Model-specific paths will be set after loading the model
SAE_MODEL_PATH = None  # Will be set to store/{model_id}/topk_sae_model.pt
METADATA_PATH = None  # Will be set to store/{model_id}/training_metadata.json

# Device configuration
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE = torch.float16 if torch.cuda.is_available() else None  # Use half precision on GPU

print("🚀 Starting TopKSAE Training Example")
print(f"📱 Using device: {DEVICE}")
print(f"🔧 Model: {MODEL_ID}")
print(f"📊 Dataset: {HF_DATASET}")
print(f"🎯 Target layer: {LAYER_SIGNATURE}")
print(f"🔢 TopK parameter: {TOP_K}")
print()

# Create output directories
CACHE_DIR.mkdir(parents=True, exist_ok=True)
STORE_DIR.mkdir(parents=True, exist_ok=True)
print("✅ Output directories created")


🚀 Starting TopKSAE Training Example
📱 Using device: cpu
🔧 Model: sshleifer/tiny-gpt2
📊 Dataset: roneneldan/TinyStories
🎯 Target layer: gpt2lmheadmodel_transformer_h_0_attn_c_attn
🔢 TopK parameter: 8

✅ Output directories created


In [24]:
# Step 1: Load language model with context
print("📥 Loading language model...")

# Load model first to get model_id
lm = LanguageModel.from_huggingface(MODEL_ID)
lm.model.to(DEVICE)

# Create model-specific directory for organizing all files
MODEL_DIR = STORE_DIR / lm.model_id
MODEL_DIR.mkdir(parents=True, exist_ok=True)

# Create store under model-specific directory (so activations are also organized by model)
store = LocalStore(MODEL_DIR)

# Set the store we want to use (overrides default)
lm.context.store = store

# Update paths to use model-specific directory
SAE_MODEL_PATH = MODEL_DIR / "topk_sae_model.pt"
METADATA_PATH = MODEL_DIR / "training_metadata.json"

# Print available layers for reference
print("🔍 Available layers:")
lm.layers.print_layer_names()
print(f"✅ Model loaded: {lm.model_id}")
print(f"📱 Device: {DEVICE}")
print(f"📁 Store base: {STORE_DIR}")
print(f"📁 Model directory: {MODEL_DIR}")
print(f"📁 Store location: {lm.context.store.base_path}")
print(f"💾 SAE model will be saved to: {SAE_MODEL_PATH}")
print(f"💾 Metadata will be saved to: {METADATA_PATH}")


📥 Loading language model...
🔍 Available layers:
gpt2lmheadmodel_transformer: No weight
gpt2lmheadmodel_transformer_wte: torch.Size([50257, 2])
gpt2lmheadmodel_transformer_wpe: torch.Size([1024, 2])
gpt2lmheadmodel_transformer_drop: No weight
gpt2lmheadmodel_transformer_h: No weight
gpt2lmheadmodel_transformer_h_0: No weight
gpt2lmheadmodel_transformer_h_0_ln_1: torch.Size([2])
gpt2lmheadmodel_transformer_h_0_attn: No weight
gpt2lmheadmodel_transformer_h_0_attn_c_attn: torch.Size([2, 6])
gpt2lmheadmodel_transformer_h_0_attn_c_proj: torch.Size([2, 2])
gpt2lmheadmodel_transformer_h_0_attn_attn_dropout: No weight
gpt2lmheadmodel_transformer_h_0_attn_resid_dropout: No weight
gpt2lmheadmodel_transformer_h_0_ln_2: torch.Size([2])
gpt2lmheadmodel_transformer_h_0_mlp: No weight
gpt2lmheadmodel_transformer_h_0_mlp_c_fc: torch.Size([2, 8])
gpt2lmheadmodel_transformer_h_0_mlp_c_proj: torch.Size([8, 2])
gpt2lmheadmodel_transformer_h_0_mlp_act: No weight
gpt2lmheadmodel_transformer_h_0_mlp_dropout: 

In [25]:
# Step 2: Load dataset
print("📥 Loading dataset...")
dataset = TextDataset.from_huggingface(
    HF_DATASET,
    split=DATA_SPLIT,
    cache_dir=str(CACHE_DIR),
    text_field=TEXT_FIELD,
    limit=DATA_LIMIT,
)
print(f"✅ Loaded {len(dataset)} text samples")


📥 Loading dataset...


Saving the dataset (1/1 shards): 100%|██████████| 1000/1000 [00:00<00:00, 380954.04 examples/s]

✅ Loaded 1000 text samples


In [26]:
# Step 3: Save activations
print("💾 Saving activations...")

# Use the store that was set on the language model
lm.activations.infer_and_save(
    dataset,
    layer_signature=LAYER_SIGNATURE,
    run_name=RUN_ID,
    store=lm.context.store,  # Use the store from language model
    batch_size=BATCH_SIZE_SAVE,
    autocast=False,  # Disable autocast for consistency
)

# Verify activations were saved
batches = lm.context.store.list_run_batches(RUN_ID)
print(f"✅ Saved {len(batches)} batches of activations")
print(f"📁 Run ID: {RUN_ID}")
print(f"📁 Store location: {lm.context.store.base_path}")


💾 Saving activations...
✅ Saved 63 batches of activations
📁 Run ID: topk_sae_training_20251110_215054
📁 Store location: store/sshleifer_tiny-gpt2


In [27]:
# Step 4: Create TopKSAE model
print("🏗️ Creating TopKSAE model...")

# Get the hidden dimension from the first batch
first_batch = lm.context.store.get_run_batch(RUN_ID, 0)
if isinstance(first_batch, dict):
    activations = first_batch["activations"]
else:
    activations = first_batch[0]  # Assume first tensor is activations

hidden_dim = activations.shape[-1]  # Last dimension is hidden size
print(f"📏 Hidden dimension: {hidden_dim}")

sae = TopKSae(
    n_latents=hidden_dim * 4,
    n_inputs=hidden_dim,
    k=TOP_K,
    device=DEVICE,
)

print(f"🧠 TopKSAE architecture: {hidden_dim} → {sae.context.n_latents} → {hidden_dim}")
print(f"🔢 TopK parameter: {sae.k}")
print(f"🔧 Device: {DEVICE}")


🏗️ Creating TopKSAE model...
📏 Hidden dimension: 6
🧠 TopKSAE architecture: 6 → 24 → 6
🔢 TopK parameter: 8
🔧 Device: cpu


In [28]:
# Step 5: Train TopKSAE using SaeTrainer
print("🏋️ Training TopKSAE...")
print("📝 Note: Training uses overcomplete's train_sae functions via the SaeTrainer composite class")
print(f"🔧 Trainer available at: sae.trainer (type: {type(sae.trainer).__name__})")
print()

# Configure training parameters
# Note: TopKSAETrainingConfig is an alias for SaeTrainingConfig
# You can also use SaeTrainingConfig directly from sae_trainer module
config = TopKSAETrainingConfig(
    epochs=100,
    batch_size=BATCH_SIZE_TRAIN,
    lr=1e-3,
    l1_lambda=1e-4,  # L1 sparsity penalty
    device=DEVICE,
    dtype=DTYPE,
    max_batches_per_epoch=50,  # Limit batches per epoch for quick training
    verbose=True,  # Enable progress logging
    use_amp=True,
    amp_dtype=DTYPE,
    clip_grad=1.0,  # Gradient clipping (overcomplete parameter)
    monitoring=2,  # Detailed monitoring (0=silent, 1=basic, 2=detailed)
)

# Train using TopKSAE's train method (which delegates to sae.trainer.train())
# The trainer uses overcomplete's train_sae_amp or train_sae functions internally
history = sae.train(lm.context.store, RUN_ID, config)

print()
print("✅ Training completed!")
print(f"📈 Final loss: {history['loss'][-1]:.6f}")
print(f"📈 Final reconstruction MSE: {history['recon_mse'][-1]:.6f}")
print(f"📈 Final L1 penalty: {history['l1'][-1]:.6f}")


2025-11-10 21:51:04,882 [INFO] amber.mechanistic.sae.sae_trainer: [SaeTrainer] Starting training run_id=topk_sae_training_20251110_215054 epochs=100 batch_size=32 device=cpu use_amp=True


🏋️ Training TopKSAE...
📝 Note: Training uses overcomplete's train_sae functions via the SaeTrainer composite class
🔧 Trainer available at: sae.trainer (type: SaeTrainer)

Epoch[1/100], Loss: 0.0736, R2: -95.7907, L0: 8.0000, Dead Features: 62.5%, Time: 0.0339 seconds
Epoch[2/100], Loss: 0.0255, R2: -32.5938, L0: 8.0000, Dead Features: 58.3%, Time: 0.0315 seconds
Epoch[3/100], Loss: 0.0078, R2: -9.2731, L0: 8.0000, Dead Features: 58.3%, Time: 0.0303 seconds
Epoch[4/100], Loss: 0.0023, R2: -2.0103, L0: 8.0000, Dead Features: 58.3%, Time: 0.0323 seconds
Epoch[5/100], Loss: 0.0007, R2: 0.0273, L0: 8.0000, Dead Features: 58.3%, Time: 0.0349 seconds
Epoch[6/100], Loss: 0.0003, R2: 0.6162, L0: 8.0000, Dead Features: 62.5%, Time: 0.0331 seconds
Epoch[7/100], Loss: 0.0002, R2: 0.7070, L0: 8.0000, Dead Features: 58.3%, Time: 0.0296 seconds
Epoch[8/100], Loss: 0.0001, R2: 0.8129, L0: 8.0000, Dead Features: 62.5%, Time: 0.0296 seconds
Epoch[9/100], Loss: 0.0001, R2: 0.8143, L0: 8.0000, Dead Featur

2025-11-10 21:51:08,045 [INFO] amber.mechanistic.sae.sae_trainer: [SaeTrainer] Completed training


Epoch[100/100], Loss: 0.0000, R2: 0.9994, L0: 8.0000, Dead Features: 66.7%, Time: 0.0344 seconds

✅ Training completed!
📈 Final loss: 0.000009
📈 Final reconstruction MSE: 0.000630
📈 Final L1 penalty: 0.000000


In [35]:
# Step 6: Save trained TopKSAE
print("💾 Saving trained TopKSAE...")

# Save using TopKSAE's save method (saves overcomplete model + our metadata)
sae.save(
    name="topk_sae_model",
    path=SAE_MODEL_PATH.parent
)

print(f"✅ TopKSAE saved to: {SAE_MODEL_PATH}")


2025-11-10 21:52:22,683 [INFO] amber.mechanistic.sae.modules.topk_sae: Saved TopKSAE to store/sshleifer_tiny-gpt2/topk_sae_model.pt


💾 Saving trained TopKSAE...
✅ TopKSAE saved to: store/sshleifer_tiny-gpt2/topk_sae_model.pt


In [37]:
lm.layers.register_hook(LAYER_SIGNATURE, sae)

'206ad0d9-4afd-4259-8eb4-9848be6d92cf'